In [1]:
from SE3Lie import *
import numpy as np
from sympy import *
from pyecca.lie import so3, r3, se3
from pyecca.lie.util import DirectProduct
from pyecca.lie.so3 import Quat, Dcm, Euler, Mrp
from pyecca.lie.r3 import R3
from pyecca.lie.se3 import SE3
from pyecca.test import test_lie
import casadi as ca
from SE3Lie import se3, SE3, se3_diff_correction_inv

In [2]:
x,y,z,i,j,k = symbols('x,y,z,i,j,k', real = True)

In [3]:
input = Matrix([x,y,z,i,j,k])
input

Matrix([
[x],
[y],
[z],
[i],
[j],
[k]])

In [4]:
r = ca.DM([0.1, 0.2, 0.3])
v = ca.DM([0.1, 0.2, 0.3])
R = Dcm.from_euler(ca.DM([0.1, 0.2, 0.3]))

In [5]:
inp = ca.horzcat(np.transpose(r),np.transpose(v))

In [6]:
veeSe3 = np.transpose(inp)
veeSe3

array([[0.1],
       [0.2],
       [0.3],
       [0.1],
       [0.2],
       [0.3]])

In [7]:
eps = 1e-7 # to avoid divide by zero
x = ca.SX.sym('x')
C1 = ca.Function('a', [x], [ca.if_else(ca.fabs(x) < eps, 1 - x ** 2 / 6 + x ** 4 / 120, ca.sin(x)/x)])
C2 = ca.Function('b', [x], [ca.if_else(ca.fabs(x) < eps, 0.5 - x ** 2 / 24 + x ** 4 / 720, (1 - ca.cos(x)) / x ** 2)])
del x

In [8]:
C1

Function(a:(i0)->(o0) SXFunction)

In [9]:
C2

Function(b:(i0)->(o0) SXFunction)

In [10]:
se3.ad_matrix(veeSe3)

SX(@1=0.3, @2=-0.2, @3=-0.3, @4=0.1, @5=-0.1, @6=0.2, 
[[00, @3, @4, 00, @3, @6], 
 [@1, 00, @5, @1, 00, @5], 
 [@2, @4, 00, @2, @4, 00], 
 [00, 00, 00, 00, @3, @6], 
 [00, 00, 00, @1, 00, @5], 
 [00, 00, 00, @2, @4, 00]])

In [11]:
se3_diff_correction_inv(veeSe3)

SX(@1=0.935755, @2=0.302933, @3=-0.0680313, 
[[@1, -0.283165, 0.20525, -0.12849, -0.253513, 0.225018], 
 [@2, 0.950581, @3, 0.312817, -0.108723, -0.0383797], 
 [-0.18054, 0.127335, 0.97529, -0.165714, 0.156986, -0.0494194], 
 [00, 00, 00, @1, -0.263397, 0.210192], 
 [00, 00, 00, @2, 0.940697, @3], 
 [00, 00, 00, -0.150888, 0.3227, 0.965406]])

In [12]:
U_inv = se3_diff_correction_inv(ca.SX([1,2,3,4,5,6]))
U_inv

SX(@1=-0.422961, @2=0.880247, @3=0.424015, 
[[@1, 0.0528417, 0.811297, -1.30632, 0.189711, 0.557791], 
 [@2, -0.213016, @3, 0.510105, -1.07305, 0.560885], 
 [0.215101, 0.975619, 0.0435836, 0.281989, 0.698786, -0.653162], 
 [00, 00, 00, @1, 0.286114, 0.904606], 
 [00, 00, 00, @2, -0.399634, @3], 
 [00, 00, 00, 0.495028, 1.11352, -0.143034]])

In [14]:
U_Matrix = SE3.inv(U_inv)
U_Matrix #check for possible errors

SX(@1=-0.422961, @2=0.880247, @3=0.424015, 
[[@1, @2, 0.215101, 00, 00, 00], 
 [0.0528417, -0.213016, 0.975619, 00, 00, 00], 
 [0.811297, @3, 0.0435836, 00, 00, 00], 
 [-1.30632, 0.510105, 0.281989, @1, @2, 0.495028], 
 [0.189711, -1.07305, 0.698786, 0.286114, -0.399634, 1.11352], 
 [0.557791, 0.560885, -0.653162, 0.904606, @3, -0.143034]])

In [15]:
ca.inv(U_inv) # U Matrix

SX(
[[-0.461953, 0.861971, 0.213222, 0.483027, -1.1546, 0.237054], 
 [0.0577131, -0.210733, 0.975853, 0.435124, 0.525225, -0.748604], 
 [0.988001, 0.463105, 0.0476016, -0.940595, 0.378782, 0.625699], 
 [-7.87996e-17, 1.12763e-16, -8.24972e-18, -0.310334, 0.783874, 0.361065], 
 [3.93527e-17, -8.86996e-17, 1.57707e-17, 0.25112, -0.289634, 0.729582], 
 [-1.07739e-16, 1.70115e-17, 2.12643e-17, 0.880927, 0.458118, -0.0619352]])

cos(x)
